In [2]:
import pandas as pd
from datetime import datetime, timezone
from sqlalchemy import create_engine
from uuid import uuid4

In [3]:
# Fill in your DB info
user_name = "postgres"
password = "password"
db_name = "postgres"
db_ip = "localhost"
port = "5432"
# Connect to the database
engine = create_engine(f'postgresql://{user_name}:{password}@{db_ip}:{port}/{db_name}')

In [4]:
# Create df_rotation, since no equivalent exists
rotation_id = uuid4()
df_rotation = pd.DataFrame(
    data=[("CTF", datetime.now(timezone.utc), rotation_id, False)],
    columns=['name', 'created_at', 'id', 'is_random'],
)
# Load each csv file into a DataFrame
df_queue = pd.read_csv('queue.csv', keep_default_na=False)
df_queue_region = pd.read_csv('queue_region.csv', keep_default_na=False)
df_player = pd.read_csv('player.csv',  keep_default_na=False)
df_player_region_trueskill = pd.read_csv('player_region_trueskill.csv', keep_default_na=False)
df_finished_game = pd.read_csv('finished_game.csv', keep_default_na=False)
df_finished_game_player = pd.read_csv('finished_game_player.csv', keep_default_na=False)
df_map = pd.read_csv('map.csv', keep_default_na=False)

FileNotFoundError: [Errno 2] No such file or directory: 'queue.csv'

In [5]:
# rotation
df_rotation

,name,created_at,id,is_random
0,CTF,2024-04-18 18:27:43.102260+00:00,1b33ae9a-c237-4428-ac32-3e69b3b158cb,False


In [ ]:
# map
rotation_index = df_map['rotation_index']
# drop the extra columns after each rotation_map has been created, since we need them for creating rotation_map entries
df_map['created_at'] = datetime.now(timezone.utc)
df_map

In [ ]:
# Create df_rotation_map, since no equivalent exists
data = []
columns = [
    'created_at',
    'id',
    'raffle_ticket_reward',
    'is_random',
    'random_probability',
    'ordinal',
    'rotation_id',
    'map_id',
    'is_next',
    'updated_at',
]
for i, row in df_map.iterrows():
    data.append(
        (
            datetime.now(timezone.utc),
            uuid4(),
            0,
            False,
            0,
            row['rotation_index'],
            df_rotation['id'].values[0],
            row['id'],
            True if row['full_name'] == 'Elite' else False,
            datetime.now(timezone.utc),
        )
    )
df_rotation_map = pd.DataFrame(
    data=data,
    columns=columns,
)
df_rotation_map

In [ ]:
# map (drop the extra columns)
df_map = df_map.drop(
    labels=[
        'rotation_index', 
        'rotation_weight',
        'is_votable',
    ],
    axis=1,
    errors='ignore',
)
df_map

In [ ]:
# queue_region -> category
df_queue_region['is_rated'] = True
df_queue_region['created_at'] = datetime.now(timezone.utc)
df_queue_region

In [ ]:
# Migrate Queue
df_queue.rename(columns={"queue_region_id": "category_id"}, inplace=True)
df_queue['is_sweaty'] = False
df_queue['mu_max'] = None
df_queue['mu_min'] = None
df_queue['ordinal'] = 0
df_queue['rotation_id'] = rotation_id
df_queue['move_enabled'] = False
df_queue['currency_award'] = None
df_queue['vote_threshold'] = None
df_queue.loc[df_queue['name'] == 'NA', ['ordinal']] = 1
df_queue.loc[df_queue['name'] == 'EU', ['ordinal']] = 2
df_queue.loc[df_queue['name'] == 'NaWest', ['ordinal']] = 3
df_queue

In [ ]:
# player
df_player['raffle_tickets'] = 0
df_player['leaderboard_enabled'] = True
df_player['stats_enabled'] = True
df_player['move_enabled'] = False
df_player['currency'] = 0
df_player = df_player.drop(labels=['unrated_trueskill_mu', 'unrated_trueskill_sigma'], axis=1)
df_player

In [ ]:
df_player_region_trueskill

In [ ]:
# player_region_trueskill -> player_category_trueskill
df_player_region_trueskill = df_player_region_trueskill.drop(
    labels=[
        'unrated_trueskill_mu', 
        'unrated_trueskill_sigma',
    ],
    axis=1,
    errors='ignore',
)
df_player_region_trueskill['rank'] = df_player_region_trueskill['rated_trueskill_mu'] - (3 * df_player_region_trueskill['rated_trueskill_sigma'])
df_player_region_trueskill.rename(
    columns={
        "queue_region_id": "category_id",
        "rated_trueskill_mu": "mu",
        "rated_trueskill_sigma": "sigma",
    },
    inplace=True,
)
df_player_region_trueskill

In [ ]:
# finished_game
df_finished_game.rename(columns={"queue_region_name": "category_name"}, inplace=True)
df_finished_game

In [ ]:
# finished_game_player
df_finished_game_player = df_finished_game_player.drop(
    labels=[
        'unrated_trueskill_mu_before', 
        'unrated_trueskill_sigma_before',
        'unrated_trueskill_mu_after', 
        'unrated_trueskill_sigma_after',
    ],
    axis=1,
    errors='ignore',
)
df_finished_game_player

In [ ]:
# Ensure the corresponding tables are clean
with engine.connect() as con:
    con.execute('DELETE FROM finished_game_player')
    con.execute('DELETE FROM finished_game')
    con.execute('DELETE FROM player_category_trueskill')
    con.execute('DELETE FROM player')
    con.execute('DELETE FROM queue')
    con.execute('DELETE FROM category')
    con.execute('DELETE FROM rotation_map')
    con.execute('DELETE FROM map')
    con.execute('DELETE FROM rotation')

In [ ]:
# Insert dataframes into the DB
df_queue_region.to_sql("category", engine, if_exists="append", index=False)
df_map.to_sql("map", engine, if_exists="append", index=False)
df_rotation.to_sql("rotation", engine, if_exists="append", index=False)
df_rotation_map.to_sql("rotation_map", engine, if_exists="append", index=False)
df_queue.to_sql("queue", engine, if_exists="append", index=False)
df_player.to_sql("player", engine, if_exists="append", index=False)
df_player_region_trueskill.to_sql("player_category_trueskill", engine, if_exists="append", index=False)
df_finished_game.to_sql("finished_game", engine, if_exists="append", index=False)
df_finished_game_player.to_sql("finished_game_player", engine, if_exists="append", index=False)